In [1]:
import requests
import pandas as pd
import glob
import os

In [2]:
# PROVINCIAL SOLO LIMA METROPOLITANA
url = "https://api.resultadoserm2022.onpe.gob.pe/results/03/140100"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
}

resp = requests.get(url, headers=headers)

data = resp.json()['results']

fecha = resp.json()['generals']['actData']['FECHA']
hora = resp.json()['generals']['actData']['HORA']
avance = resp.json()['generals']['actData']['POR_AVANCE']

if data != []:
    df = pd.DataFrame(data)
    df['fecha'] = fecha
    df['hora'] = hora
    df['avance'] = avance
    df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
    df = df.dropna(subset=['C_CODI_AGP'])
    df.to_csv(f"lima_metropolitana.csv")

In [3]:
# REGIONAL
try:
    for departamento in range(1,26):
        url = f"https://api.resultadoserm2022.onpe.gob.pe/results/01/{departamento:02d}0000"
        headers = {
            "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
        }

        resp = requests.get(url, headers=headers)

        data = resp.json()['results']

        fecha = resp.json()['generals']['actData']['FECHA']
        hora = resp.json()['generals']['actData']['HORA']
        avance = resp.json()['generals']['actData']['POR_AVANCE']

        if data != []:
            df = pd.DataFrame(data)
            df['cod_departamento'] = f'{departamento:02d}'
            df['fecha'] = fecha
            df['hora'] = hora
            df['avance'] = avance
            df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
            df = df.dropna(subset=['C_CODI_AGP'])
            df.to_csv(f"resultados_regionales/{departamento:02d}0000.csv")
except:
    pass

In [4]:
# setting the path for joining multiple files
files = os.path.join("resultados_regionales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0, converters = {'cod_departamento': str.strip, 'avance': str.strip})
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True).drop(columns=['Unnamed: 0'])
frame = frame.sort_values(by=['cod_departamento', 'POR_VALIDOS'])
frame = frame.reset_index(drop=True)

dict_regiones = {
    "01":"AMAZONAS",
    "02":"ÁNCASH",
    "03":"APURÍMAC",
    "04":"AREQUIPA",
    "05":"AYACUCHO",
    "06":"CAJAMARCA",
    "07":"CUSCO",
    "08":"HUANCAVELICA",
    "09":"HUÁNUCO",
    "10":"ICA",
    "11":"JUNÍN",
    "12":"LA LIBERTAD",
    "13":"LAMBAYEQUE",
    "14":"LIMA",
    "15":"LORETO",
    "16":"MADRE DE DIOS",
    "17":"MOQUEGUA",
    "18":"PASCO",
    "19":"PIURA",
    "20":"PUNO",
    "21":"SAN MARTÍN",
    "22":"TACNA",
    "23":"TUMBES",
    "24":"CALLAO",
    "25":"UCAYALI",
}

frame['departamento'] = frame['cod_departamento'].map(dict_regiones)


Resultant CSV after joining all CSV files at a particular location...


In [5]:
# agregando nombres de candidatos
df_candidatos = pd.read_csv('../candidatos/regionales_total.csv')
df_candidatos


dict_tildes = {
    "AMAZONAS":"AMAZONAS",
    "ANCASH":"ÁNCASH",
    "APURIMAC":"APURÍMAC",
    "AREQUIPA":"AREQUIPA",
    "AYACUCHO":"AYACUCHO",
    "CAJAMARCA":"CAJAMARCA",
    "CUSCO":"CUSCO",
    "HUANCAVELICA":"HUANCAVELICA",
    "HUANUCO":"HUÁNUCO",
    "ICA":"ICA",
    "JUNIN":"JUNÍN",
    "LA LIBERTAD":"LA LIBERTAD",
    "LAMBAYEQUE":"LAMBAYEQUE",
    "LIMA":"LIMA",
    "LORETO":"LORETO",
    "MADRE DE DIOS":"MADRE DE DIOS",
    "MOQUEGUA":"MOQUEGUA",
    "PASCO":"PASCO",
    "PIURA":"PIURA",
    "PUNO":"PUNO",
    "SAN MARTIN":"SAN MARTÍN",
    "TACNA":"TACNA",
    "TUMBES":"TUMBES",
    "CALLAO":"CALLAO",
    "UCAYALI":"UCAYALI",
}
dict_correcciones_partidos = {
    "VICTORIA AMAZONENSE":"MOVIMIENTO REGIONAL VICTORIA AMAZONENSE",
    "ALIANZA GOBIERNO UNIDAD Y ACCION":"ALIANZA GOBIERNO UNIDAD Y ACCION - AGUA",
    "HATARIY APURIMAC":"MOVIMIENTO REGIONAL HATARIY APURIMAC",
    "TRABAJO MAS TRABAJO":"MOVIMIENTO REGIONAL TRABAJO MAS TRABAJO",
    "MOVIMIENTO CIVICO PERUANO":"NUEVA ESPERANZA",
    "MOVIMENTO INDEPENDIENTE REGIONAL FUERZA TACNA":"MOVIMIENTO INDEPENDIENTE REGIONAL FUERZA TACNA"
}
df_candidatos['postulaDepartamento'] = df_candidatos['postulaDepartamento'].map(dict_tildes)
df_candidatos['organizacionPolitica'] = df_candidatos['organizacionPolitica'].replace(dict_correcciones_partidos)


df_final = pd.merge(frame, df_candidatos[['postulaDepartamento', 'organizacionPolitica', 'nombreCompleto']],  how='left', left_on=['departamento','AGRUPACION'], right_on = ['postulaDepartamento','organizacionPolitica'])
df_final.to_csv('resultados_regional.csv')